# 1. Thông tin nhóm

### *Danh sách thành viên:*

<center>
    
|   MSSV   |      Họ tên      | Tài khoản Github |
|:--------:|:----------------:|:----------------:|
| 19120119 | Nguyễn Minh Phúc |   youngtobesick  |
| 19120387 |    Lê Sỹ Thuần   |    SyThuan1803   |
    
</center>

### *Kế hoạch làm việc:* 
[Xem tại đây](https://docs.google.com/spreadsheets/d/1DCq30ORIH6m-1EYMnbUmZ8fBoxPiiCgRgLDpJaRKRVE/edit?usp=sharing)

---

# 2. Thu thập dữ liệu

 **Dữ liệu của bạn là về chủ đề gì và bạn lấy từ nguồn nào?**

Dữ liệu là bảng đánh giá độ hài lòng của các khách về các khách sạn Châu Âu
</br> Dữ liệu được lấy trên Kaggle: https://www.kaggle.com/ishansingh88/europe-hotel-satisfaction-score?fbclid=IwAR3WJzG0H6I6zSR_53owrMgq_oswiWWCD9lsc985izQLlxJo0L2HDauUlw8


**Phần mô tả của dữ liệu**
Dữ liệu được lấy sau khi khách hàng thực hiện khảo sát bằng cách chấm điểm từ 1 đến 5 theo các tiêu chí được đưa ra dành cho khách sạn và kết luận hài lòng hay không hài lòng

**License của dữ liệu**

---

# 3. Import các thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np

---

# 4. Khám phá & tiền xử lý dữ liệu

## Khám phá dữ liệu

### Đọc dữ liệu từ file csv vào dataframe

In [2]:
df=pd.read_csv('Europe Hotel Booking Satisfaction Score.csv')
df

,id,Gender,Age,purpose_of_travel,Type of Travel,Type Of Booking,Hotel wifi service,Departure/Arrival convenience,Ease of Online booking,Hotel location,Food and drink,Stay comfort,Common Room entertainment,Checkin/Checkout service,Other service,Cleanliness,satisfaction
0,70172,Male,13,aviation,Personal Travel,Not defined,3,4,3,1,5,5,5,4,5,5,neutral or dissatisfied
1,5047,Male,25,tourism,Group Travel,Group bookings,3,2,3,3,1,1,1,1,4,1,neutral or dissatisfied
2,110028,Female,26,tourism,Group Travel,Group bookings,2,2,2,2,5,5,5,4,4,5,satisfied
3,24026,Female,25,tourism,Group Travel,Group bookings,2,5,5,5,2,2,2,1,4,2,neutral or dissatisfied
4,119299,Male,61,aviation,Group Travel,Group bookings,3,3,3,3,4,5,3,3,3,3,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103899,94171,Female,23,business,Group Travel,Individual/Couple,2,1,2,3,2,2,2,2,3,2,neutral or dissatisfied
103900,73097,Male,49,tourism,Group Travel,Group bookings,4,4,4,4,2,5,5,5,5,4,satisfied
103901,68825,Male,30,tourism,Group Travel,Group bookings,1,1,1,3,4,5,4,5,5,4,neutral or dissatisfied
103902,54173,Female,22,business,Group Travel,Individual/Couple,1,1,1,5,1,1,1,5,4,1,neutral or dissatisfied


### Dữ liệu gồm có bao nhiêu dòng và bao nhiêu cột?

In [3]:
nrow,ncol=df.shape
print(f'Số dòng: {nrow}')
print(f'Số cột: {ncol}')

Số dòng: 103904
Số cột: 17


### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng cho biết thông tin một phiếu đánh giá của một khách hàng. Các dòng không có ý nghĩa gì đặc biệt

### Dữ liệu có các dòng bị lặp không? (0.5đ)

In [4]:
print(df.duplicated().sum())

0


Dữ liệu không có dòng nào bị lặp

### Mỗi cột có ý nghĩa gì?

|Thuộc tính|    Ý nghĩa   |
|:--------:|:-----:|
|**id**|ID của survey||
|**Gender**|Giới tính người thực hiện survey|
|**Age**|Tuổi| 
|**purpose_of_travel**|Mục đích chuyến đi|
|**type_of_travel**|Loại chuyến đi|
|**Hotel wifi**|Wifi của khách sạn|
|**Departure/Arrival  convenience**|Tiện nghi khi đến|
|**Ease of Online booking**|Thoải mái booking online|
|**Hotel location**|Địa điểm hotel|
|**Food and drink**|Thức ăn và đồ uống|
|**Stay comfort**|Chỗ ở thoải mái|
|**Common Room entertainment**|Phòng giải trí chung|
|**Checkin/Checkout service**|Dịch vụ check in/ check out|
|**Other service**|Dịch vụ khác|
|**Cleanliness**|Sạch sẽ|
|**satisfaction**|Độ hài lòng|

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

Nhìn sơ qua về kiểu dữ liệu của các cột

In [5]:
df.dtypes

id                                 int64
Gender                            object
Age                                int64
purpose_of_travel                 object
Type of Travel                    object
Type Of Booking                   object
Hotel wifi service                 int64
Departure/Arrival  convenience     int64
Ease of Online booking             int64
Hotel location                     int64
Food and drink                     int64
Stay comfort                       int64
Common Room entertainment          int64
Checkin/Checkout service           int64
Other service                      int64
Cleanliness                        int64
satisfaction                      object
dtype: object

Ta nên đổi cột id về object

In [6]:
df['id']=df['id'].astype('object')

**Phân tích dữ liệu các cột dạng số**

In [7]:
numeric_df=df.select_dtypes(include=["int64"])

Các thông số thống kê

In [8]:
summary=numeric_df.describe().T
summary['count']=(len(df)-summary['count'])/len(df)*100
summary.rename(columns={'count':'missing ratio'},inplace=True)
summary

,missing ratio,mean,std,min,25%,50%,75%,max
Age,0.0,39.379706,15.114964,7.0,27.0,40.0,51.0,85.0
Hotel wifi service,0.0,2.729683,1.327829,0.0,2.0,3.0,4.0,5.0
Departure/Arrival convenience,0.0,3.060296,1.525075,0.0,2.0,3.0,4.0,5.0
Ease of Online booking,0.0,2.756901,1.398929,0.0,2.0,3.0,4.0,5.0
Hotel location,0.0,2.976883,1.277621,0.0,2.0,3.0,4.0,5.0
Food and drink,0.0,3.202129,1.329533,0.0,2.0,3.0,4.0,5.0
Stay comfort,0.0,3.439396,1.319088,0.0,2.0,4.0,5.0,5.0
Common Room entertainment,0.0,3.358158,1.332991,0.0,2.0,4.0,4.0,5.0
Checkin/Checkout service,0.0,3.304290,1.265396,0.0,3.0,3.0,4.0,5.0
Other service,0.0,3.640428,1.175663,0.0,3.0,4.0,5.0,5.0


**Nhận xét các dữ liệu số:**
- Độ tuổi trung bình của người thực hiện khảo sát là 39 tuổi, nhỏ nhất là 7 tuổi, lớn nhất là 85 tuổi 
- Các chỉ số đánh giá mức độ hài lòng từ 0 đến 5
- Các giá trị trung bình của chỉ số đánh giá nằm từ 2 đến 3, nhìn chung cũng khá cân bằng

**Phân tích dữ liệu dạng object**

In [9]:
categorical=df.select_dtypes(include="object")
categorical.describe()

,id,Gender,purpose_of_travel,Type of Travel,Type Of Booking,satisfaction
count,103904,103904,103904,103904,103904,103904
unique,103904,2,5,2,3,2
top,1,Female,tourism,Group Travel,Group bookings,neutral or dissatisfied
freq,1,52727,32053,71655,49665,58879


In [10]:

cate_columns=list(categorical.columns)
missing_ratio=df[cate_columns].isnull().sum()/len(df)*100
missing_ratio=missing_ratio.to_frame().transpose()
num_diff_vals=df[cate_columns].nunique().to_frame().transpose()
diff_vals=df[cate_columns].apply(lambda col: col.dropna().unique()).to_frame().transpose()
cate_col_profiles_df=pd.concat([missing_ratio,num_diff_vals,diff_vals]).reset_index().drop('index',1)
cate_col_profiles_df.rename(index={0:"missing_ratio",1:"num_diff_vals", 2:"diff_vals"},inplace=True)

In [11]:
cate_col_profiles_df

,id,Gender,purpose_of_travel,Type of Travel,Type Of Booking,satisfaction
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0
num_diff_vals,103904,2,5,2,3,2
diff_vals,"[70172, 5047, 110028, 24026, 119299, 111157, 8...","[Male, Female]","[aviation, tourism, business, academic, personal]","[Personal Travel, Group Travel]","[Not defined, Group bookings, Individual/Couple]","[neutral or dissatisfied, satisfied]"


**Nhận xét:**
- Gender: có 2 loại nam, nữ
- Purpose_of_travel: có 5 giá trị aviation(hàng không), tourism(theo tour), business(công việc), academic(trường học), personal(cá nhân)
- Type of travel: Personal Travel(cá nhân), Group Travel(theo nhóm)
- Type of booking: Not defined(không xác định), Group bookings(theo nhóm), Individual/Couple(cá nhân/ cặp đôi)
- Satisfaction: neutral or dissatisfied(bình thường hoặc không hài lòng), satisfied(hài lòng)

**Vấn đề cần tiền xử lý:**
- Cột Type Of Booking có giá trị là Not defined tương tự với việc đánh dấu giá trị thiếu, nên cần phải điền vào những ô có giá trị này.

---

## Tiền xử lý

**Thay thế những ô có giá trị Not defined trong cột Type of booking**

**Nhận xét:**
- Vì xét theo nghĩa của type of booking thì với giá trị thuộc tính Type of travel là personal travel sẽ có Type of booking là Individual/ Couple
- Nên ta có thể suy ra theo thuộc tính Type of travel với giá trị là Personal Travel

In [12]:
df.loc[df['Type Of Booking'] == 'Not defined', 'Type Of Booking'] = np.nan

In [13]:
df['Gender'] = df['Gender'].replace(['Male'],1)
df['Gender'] = df['Gender'].replace(['Female'],2)

df['Type of Travel'] = df['Type of Travel'].replace(['Personal Travel'],1)
df['Type of Travel'] = df['Type of Travel'].replace(['Group Travel'],2)

df['satisfaction'] = df['satisfaction'].replace(['neutral or dissatisfied'],2)
df['satisfaction'] = df['satisfaction'].replace(['satisfied'],'1')
df['Type Of Booking'] = df['Type Of Booking'].replace(['Individual/Couple'],1)
#df['Type Of Booking'] = df['Type Of Booking'].replace(['Group bookings'],2')
df.loc[df['Type Of Booking'] == 'Group bookings', 'Type Of Booking'] = 2
df['purpose_of_travel'] = df['purpose_of_travel'].replace(['tourism'],5)
df['purpose_of_travel'] = df['purpose_of_travel'].replace(['academic'],4)
df['purpose_of_travel'] = df['purpose_of_travel'].replace(['business'],3)
df['purpose_of_travel'] = df['purpose_of_travel'].replace(['aviation'],2)
df['purpose_of_travel'] = df['purpose_of_travel'].replace(['personal'],1)


In [14]:
sample_df=df[~df['Type Of Booking'].isna()]
sample_df=sample_df.sample(frac=0.01, random_state=30).reset_index(drop=True)
sample_df_isnan=df[df['Type Of Booking'].isna()].sample(frac=0.028, random_state=30).reset_index(drop=True)
sample_df1=pd.concat((sample_df_isnan,sample_df),axis=0).reset_index(drop=True)
sample_df1

,id,Gender,Age,purpose_of_travel,Type of Travel,Type Of Booking,Hotel wifi service,Departure/Arrival convenience,Ease of Online booking,Hotel location,Food and drink,Stay comfort,Common Room entertainment,Checkin/Checkout service,Other service,Cleanliness,satisfaction
0,51458,2,21,4,1,NaN,3,4,3,3,1,1,1,4,3,1,2
1,14664,1,17,4,2,NaN,5,5,5,5,5,3,5,5,2,5,1
2,22020,2,37,5,2,NaN,4,5,5,5,4,4,4,2,1,4,1
3,55187,1,54,3,2,NaN,4,5,4,5,4,4,4,1,2,4,2
4,11522,2,42,4,1,NaN,3,2,3,4,3,4,5,3,5,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169,61001,2,45,5,2,1,2,1,2,3,5,5,5,1,2,5,2
1170,112706,1,39,5,2,1,4,4,4,3,1,1,1,1,4,1,2
1171,2663,2,53,3,2,2,5,5,5,5,3,5,4,5,4,5,1
1172,46666,1,30,4,2,2,4,1,1,1,2,4,2,1,3,2,2


In [15]:
ratings=sample_df1.drop('id',axis=1)
ratings['Type Of Booking']=pd.to_numeric(ratings['Type Of Booking'])
ratings=ratings.astype('float64')
ratings=np.array(ratings.iloc[:,:])
ratings

array([[ 2., 21.,  4., ...,  3.,  1.,  2.],
       [ 1., 17.,  4., ...,  2.,  5.,  1.],
       [ 2., 37.,  5., ...,  1.,  4.,  1.],
       ...,
       [ 2., 53.,  3., ...,  4.,  5.,  1.],
       [ 1., 30.,  4., ...,  3.,  2.,  2.],
       [ 2., 23.,  4., ...,  4.,  5.,  1.]])

In [16]:
batch_size = 32
filled_ratings = np.empty_like(ratings)
start = 0
end = batch_size
new_ratings=np.tile(ratings,(batch_size,1))
while(1):
    if (start>=len(ratings)):
        break
    r=ratings[start:end,:]
    if batch_size != 32:
        new_ratings=np.tile(ratings,(batch_size,1))
    r= np.repeat(r, repeats=len(ratings), axis=0)
    a=np.abs(new_ratings[:,:] -r)
    #Tao similarities
    similarities=np.zeros((batch_size,len(ratings)))
    startRow=0
    for ind in range(batch_size):
        similarities[ind]=(1/(np.nanmean(a[startRow:startRow+len(ratings)],axis=1)+0.001))
        startRow+=len(ratings)
    similarities=np.nan_to_num(similarities,nan=0) 
    weights=np.zeros((batch_size,len(ratings),ratings.shape[1]))
    for ind in range(batch_size):
        normalize_w=~np.isnan(ratings)*similarities[ind].reshape(-1,1)
        weights[ind]=normalize_w/normalize_w.sum(axis=0)
    filled_ratingsBatch=np.nansum(ratings*weights,axis=1)
    filled_ratings[start:end]=filled_ratingsBatch
    start=end
    end=start+batch_size
    if(end>len(ratings)):
        end=len(ratings)
        batch_size=end-start

In [17]:
filled_nanvals = filled_ratings[np.isnan(ratings)].round(0)
sample_df_isnan['Type Of Booking']=filled_nanvals
# sample_df=pd.concat((sample_df,sample_df_isnan),axis=0).reset_index(drop=True)
sample_df=pd.concat((sample_df_isnan,sample_df),ignore_index=True,axis=0)
sample_df

,id,Gender,Age,purpose_of_travel,Type of Travel,Type Of Booking,Hotel wifi service,Departure/Arrival convenience,Ease of Online booking,Hotel location,Food and drink,Stay comfort,Common Room entertainment,Checkin/Checkout service,Other service,Cleanliness,satisfaction
0,51458,2,21,4,1,1.0,3,4,3,3,1,1,1,4,3,1,2
1,14664,1,17,4,2,1.0,5,5,5,5,5,3,5,5,2,5,1
2,22020,2,37,5,2,2.0,4,5,5,5,4,4,4,2,1,4,1
3,55187,1,54,3,2,2.0,4,5,4,5,4,4,4,1,2,4,2
4,11522,2,42,4,1,2.0,3,2,3,4,3,4,5,3,5,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169,61001,2,45,5,2,1,2,1,2,3,5,5,5,1,2,5,2
1170,112706,1,39,5,2,1,4,4,4,3,1,1,1,1,4,1,2
1171,2663,2,53,3,2,2,5,5,5,5,3,5,4,5,4,5,1
1172,46666,1,30,4,2,2,4,1,1,1,2,4,2,1,3,2,2


---

## Khám phá dữ liệu (tiếp tục)

### Với mỗi cột có kiểu dữ liệu dạng numeric, các giá trị được phân bố như thế nào?

### Với mỗi cột có kiểu dữ liệu dạng categorical, các giá trị được phân bố như thế nào?

---

# 5. Đưa ra các câu hỏi cần trả lời

---

# 6.Tiền xử lý, phân tích dữ liệu để trả lời cho từng câu hỏi

## 6.1 Câu hỏi 1

### Tiền xử lý

### Phân tích dữ liệu để trả lời cho các câu hỏi 

---

## 6.2 Câu hỏi 2

### Tiền xử lý

### Phân tích dữ liệu để trả lời cho các câu hỏi 

---

# 7. Nhìn lại quá trình làm đồ án

## Thành viên 1

- Họ tên:
- Đã gặp những khó khăn gì?
- Có học được gì hữu ích?

## Thành viên 2

- Họ tên:
- Đã gặp những khó khăn gì?
- Có học được gì hữu ích?

## Nhóm
**Nếu có thêm thời gian thì sẽ làm gì?**

# 8. Tài liệu tham khảo